# MScBMI 33200 – Assignment III
### Savita K Gupta
### 17 May 2023

In [ ]:
#Imports

import numpy as np
import pandas as pd
from scipy.stats import sem
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier 

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.neural_network import MLPClassifier

## Section 1: ER Bots 30-Day Readmission Study

### S1 Question 1: Naive Model

In [ ]:
# Import train and test dataset

rm_train_full = pd.read_csv(r'C:\Users\vitak\Downloads\readmission_train.csv')
rm_test_full = pd.read_csv(r'C:\Users\vitak\Downloads\readmission_test.csv')

In [ ]:
# Setup xTrain and yTrain

rm_n = rm_train_full.drop(rm_train_full.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]], axis=1)

rm_Xtrain_n = rm_n.drop(["outcome"], axis=1)
rm_Ytrain_n = rm_n['outcome']

In [ ]:
#Setup xTest and yTest

rm_n_test = rm_test_full.drop(rm_train_full.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]], axis=1)

rm_Xtest_n = rm_n_test.drop(["outcome"], axis=1)
rm_Ytest_n = rm_n_test['outcome']

In [ ]:
# Run logistic regression

naive_model = LogisticRegression()
naive_model.fit(rm_Xtrain_n, rm_Ytrain_n)

In [ ]:
#Calculate AUC using predict_proba

naive_model_prediction = naive_model.predict_proba(rm_Xtest_n)
print("Naive Model train set AUC score - with predict proba: %f" % roc_auc_score(rm_Ytest_n, naive_model_prediction[:,1]))

In [ ]:
# Calculate Confidence Interval using bootstrap for Naive model

#Y train value for Naive model
y_true_n = np.array(rm_Ytest_n)

#Predict_proba value for Naive model
y_pred_n = np.array(naive_model_prediction[:,1])


n_bootstraps = 100
rng_seed = 42  # control reproducibility
bootstrapped_scores = []


rng = np.random.RandomState(rng_seed)

for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_n), len(y_pred_n))
    if len(np.unique(y_true_n[indices])) < 2:
        continue

    score = roc_auc_score(y_true_n[indices], y_pred_n[indices])
    bootstrapped_scores.append(score)
    
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

con_low_n = sorted_scores[int(0.05 * len(sorted_scores))]
con_up_n = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.5f} - {:0.5}]".format(con_low_n, con_up_n))

In [ ]:
### Cross Validation: Naive Model

In [ ]:
#Cross Validation with kfold
k = 5
rm_kf = KFold(n_splits=k, random_state=None)
rm_model = LogisticRegression(solver= 'liblinear')

result = cross_val_score(rm_model , rm_Xtest_n, rm_Ytest_n, cv = rm_kf)
print("Avg accuracy: {}".format(result.mean()))

In [ ]:
#LogisticRegressionCV (KFold=5)

rm_log_cv = LogisticRegressionCV(cv=5, random_state = 0)
rm_log_cv.fit(rm_Xtrain_n, rm_Ytrain_n)

rm_log_cv_predict = rm_log_cv.predict_proba(rm_Xtest_n)

rm_n_auc = roc_auc_score(rm_Ytest_n, rm_log_cv_predict[:,1])
print("Naive Model training AUC (with Kfold CV): %f" % rm_n_auc)

In [ ]:
# Calculate Confidence Interval using bootstrap for Naive model (with LogReg Kfold CV)

#Y train value for Naive model
y_true_n = np.array(rm_Ytest_n)

#Predict_proba value for Naive model
y_pred_n = np.array(rm_log_cv_predict[:,1])


n_bootstraps = 100
rng_seed = 42  # control reproducibility
bootstrapped_scores = []


rng = np.random.RandomState(rng_seed)

for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_n), len(y_pred_n))
    if len(np.unique(y_true_n[indices])) < 2:
        continue

    score = roc_auc_score(y_true_n[indices], y_pred_n[indices])
    bootstrapped_scores.append(score)
    
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

cv_con_low_n = sorted_scores[int(0.05 * len(sorted_scores))]
cv_con_up_n = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.5f} - {:0.5}]".format(cv_con_low_n, cv_con_up_n))

### S1 Question 2: GLM Model

In [ ]:
# Setup xTrain and yTrain

rm_lr_train = rm_train_full.drop_duplicates(keep='last')

rm_Xtrain_lr = rm_lr_train.drop(["outcome"], axis=1)
rm_Ytrain_lr = rm_lr_train['outcome']

In [ ]:
# Run logistic regression

lr_model = LogisticRegression()
lr_model.fit(rm_Xtrain_lr, rm_Ytrain_lr)

In [ ]:
#Calculate AUC

lr_model_prediction = lr_model.predict_proba(rm_Xtest_lr)
print("Logistic Regression train set AUC score: %f" % roc_auc_score(rm_Ytest_lr, lr_model_prediction[:,1]))

In [ ]:
# Calculate Confidence Interval using bootstrap for Logistic Regression model

#Y train value for Logistic Regression model
y_true_lr = np.array(rm_Ytest_lr)

#Predict_proba value for Logistic Regression model
y_pred_lr = np.array(lr_model_prediction[:,1])

n_bootstraps = 100
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)

for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_lr), len(y_pred_lr))
    if len(np.unique(y_true_lr[indices])) < 2:
        continue

    score = roc_auc_score(y_true_lr[indices], y_pred_lr[indices])
    bootstrapped_scores.append(score)
    
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

con_low_lr = sorted_scores[int(0.05 * len(sorted_scores))]
con_up_lr = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.5f} - {:0.5}]".format(con_low_lr, con_up_lr))

### Cross Validation: GLM Model

In [ ]:
#Cross Validation with kfold
k = 5
rm_kf_lr = KFold(n_splits=k, random_state=None)
rm_model_lr = LogisticRegression(solver= 'liblinear')

result_lr = cross_val_score(lr_model , rm_Xtest_lr, rm_Ytest_lr, cv = rm_kf_lr)
print("Avg accuracy: {}".format(result_lr.mean()))

In [ ]:
#LogisticRegressionCV (KFold=5)

lr_log_cv = LogisticRegressionCV(cv=5, random_state = 0)
lr_log_cv.fit(rm_Xtrain_lr, rm_Ytrain_lr)

lr_log_cv_predict = lr_log_cv.predict_proba(rm_Xtest_lr)

rm_lr_auc = roc_auc_score(rm_Ytest_lr, lr_log_cv_predict[:,1])
print("GLM Model training AUC (with Kfold CV): %f" % rm_lr_auc)

In [ ]:
# Calculate Confidence Interval using bootstrap for Logistic Regression model (with Kfold CV)

#Y train value for Naive model
y_true_lr = np.array(rm_Ytest_lr)

#Predict_proba value for Naive model
y_pred_lr = np.array(lr_log_cv_predict[:,1])


n_bootstraps = 100
rng_seed = 42  # control reproducibility
bootstrapped_scores = []


rng = np.random.RandomState(rng_seed)

for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_lr), len(y_pred_lr))
    if len(np.unique(y_true_lr[indices])) < 2:
        continue

    score = roc_auc_score(y_true_lr[indices], y_pred_lr[indices])
    bootstrapped_scores.append(score)
    
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

cv_con_low_lr = sorted_scores[int(0.05 * len(sorted_scores))]
cv_con_up_lr = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.5f} - {:0.5}]".format(cv_con_low_lr, cv_con_up_lr))

### S1 Question 3: Neural Net Classifier

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(100,80,10,1),
    activation='relu',
    max_iter=500,
    alpha=0.10,
    solver="sgd",
    verbose=10,
    random_state=1,
    learning_rate='adaptive',
    learning_rate_init=0.001,
)


mlp.fit(rm_Xtrain_lr, rm_Ytrain_lr)

mlp_predtrain = mlp.predict_proba(rm_Xtrain_lr)
print("Train set AUC score: %f" % roc_auc_score(rm_Ytrain_lr, mlp_predtrain[:,1]))


mlp_predtest = mlp.predict_proba(rm_Xtest_lr)
print("Train set AUC score: %f" % roc_auc_score(rm_Ytest_lr, mlp_predtest[:,1]))

### Cross Validation
This section must run overnight. GRIDSEARCH can take up to 8 hours.

In [ ]:
 mlp = MLPClassifier(max_iter=200)
    parameter_space = { 'hidden_layer_sizes': [(50,20,10,1),(50,10),(20,10,5), (100,80,10,1)], 
                       'activation': ['relu'], 
                       'solver': ['sgd','adam'], 
                       'batch_size': [300,500], 
                       'alpha': [0.10,0.40,0.60,0.80,1.0],
                       'random_state': [1], 
                       'learning_rate': ['constant', 'adaptive'], 
                       'learning_rate_init': [0.1, 0.01, 0.001] } 
    
    clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5) 
    
    clf.fit(rm_Xtrain_lr, rm_Ytrain_lr) 
    
    print('Best parameters found:\n', clf.best_params_)

In [ ]:
#Calculate AUC after GridSearch 

mlp = MLPClassifier(
    hidden_layer_sizes=(50,20,10,1),
    activation='relu',
    solver="sgd",
    batch_size = 300,
    alpha=0.1,
    random_state=1,
    learning_rate='constant',
    learning_rate_init=0.1,
    max_iter=500,
    verbose=10, 
)


mlp.fit(rm_Xtrain_lr, rm_Ytrain_lr)

mlp_predtrain = mlp.predict_proba(rm_Xtrain_lr)
print("Train set AUC score: %f" % roc_auc_score(rm_Ytrain_lr, mlp_predtrain[:,1]))

mlp_predtest = mlp.predict_proba(rm_Xtest_lr)
print("Test set AUC score: %f" % roc_auc_score(rm_Ytest_lr, mlp_predtest[:,1]))

In [ ]:
#Calculate Confidence Interval After Neural Net GridSearch

y_true_nn = np.array(rm_Ytest_lr)
y_pred_nn = np.array(mlp_predtest[:,1])

n_bootstraps = 100
rng_seed = 42
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_nn), len(y_pred_nn))
    if len(np.unique(y_true_nn[indices])) < 2:
        continue

    score = roc_auc_score(y_true_nn[indices], y_pred_nn[indices])
    bootstrapped_scores.append(score)
    
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower_nn = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper_nn = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.5f} - {:0.5}]".format(confidence_lower_nn, confidence_upper_nn))
Confidence interval for the score: [0.50000 - 0.5]

## Section 2: GUSTO Study

### S2 Question 1: GLM Model

In [ ]:
# Import Data

g_train = pd.read_csv(r'C:\Users\vitak\Downloads\gusto_train.csv')
g_test = pd.read_csv(r'C:\Users\vitak\Downloads\gusto_test.csv')

In [ ]:
# Setup xTrain and yTrain

g_Xtrain = g_train.drop(["DAY30"], axis=1)
g_Ytrain = g_train['DAY30']

In [ ]:
# Setup xTest and yTest

g_Xtest = g_test.drop(["DAY30"], axis=1)
g_Ytest = g_test['DAY30']

In [ ]:
# Setup xTrain and yTrain

g_Xtrain = g_train.drop(["DAY30"], axis=1)
g_Ytrain = g_train['DAY30']

In [ ]:
#Calculate AUC using predict_proba

g_rf_prediction = g_rfClassifier.predict_proba(g_Xtest)
print("Random Forest train set AUC score: %f" % roc_auc_score(g_Ytest, g_rf_prediction[:,1]))

In [ ]:
# Calculate Confidence Interval using bootstrap for GUSTO Logistic Regression model

#Y train value for Logistic Regression model
gy_true_lr = np.array(g_Ytest)

#Predict_proba value for Logistic Regression model
gy_pred_lr = np.array(g_lr_model_prediction[:,1])


n_bootstraps = 100
rng_seed = 42  # control reproducibility
bootstrapped_scores = []


rng = np.random.RandomState(rng_seed)

for i in range(n_bootstraps):
    indices = rng.randint(0, len(gy_pred_lr), len(gy_pred_lr))
    if len(np.unique(gy_true_lr[indices])) < 2:
        continue

    score = roc_auc_score(gy_true_lr[indices], gy_pred_lr[indices])
    bootstrapped_scores.append(score)
    
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

gcon_low_lr = sorted_scores[int(0.05 * len(sorted_scores))]
gcon_up_lr = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.5f} - {:0.5}]".format(gcon_low_lr, gcon_up_lr))

### Cross Validation: GUSTO GLM

In [ ]:
#Cross Validation with kfold
k = 5
g_kf = KFold(n_splits=k, random_state=None)
g_model = LogisticRegression(solver= 'liblinear')

gresult_lr = cross_val_score(g_model , g_Xtest, g_Ytest, cv = g_kf)
print("Avg accuracy: {}".format(gresult_lr.mean()))


In [ ]:
#LogisticRegressionCV (KFold=5)

g_log_cv = LogisticRegressionCV(cv=5, random_state = 0)
g_log_cv.fit(g_Xtrain, g_Ytrain)

g_log_cv_predict = g_log_cv.predict_proba(g_Xtest)

g_lr_auc = roc_auc_score(g_Ytest, g_log_cv_predict[:,1])
print("GUSTO GLM AUC (with Kfold CV): %f" % g_lr_auc)

In [ ]:
# Calculate Confidence Interval using bootstrap for GUSTO Logistic Regression model (with Kfold CV)

#Y train value for Logistic Regression model
gy_true_cv = np.array(g_Ytest)

#Predict_proba value for Logistic Regression model
gy_pred_cv = np.array(g_log_cv_predict[:,1])


n_bootstraps = 100
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)

for i in range(n_bootstraps):
    indices = rng.randint(0, len(gy_pred_cv), len(gy_pred_cv))
    if len(np.unique(gy_true_cv[indices])) < 2:
        continue

    score = roc_auc_score(gy_true_cv[indices], gy_pred_cv[indices])
    bootstrapped_scores.append(score)
    
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

gcon_low_cv = sorted_scores[int(0.05 * len(sorted_scores))]
gcon_up_cv = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.5f} - {:0.5}]".format(gcon_low_cv, gcon_up_cv))

### S2 Question 2: Ridge Regression Model

In [ ]:
#Run Ridge Regression
r_alphas = np.logspace(0,5,100)
ridge = RidgeCV(alphas = r_alphas, scoring = 'r2')
ridge.fit(g_Xtrain, g_Ytrain)

In [ ]:
#Calculate AUC
grr_prob = ridge.predict(g_Xtest)
grr_auc_roc = roc_auc_score(g_Ytest, grr_prob)
print("Ridge Regression AUC: %f" % grr_auc_roc)

In [ ]:
# Calculate Confidence Interval using bootstrap for GUSTO Ridge Regression model

#Y train value for Logistic Regression model
grr_true = np.array(g_Ytest)

#Predict_proba value for Logistic Regression model
grr_pred = np.array(grr_prob)


n_bootstraps = 100
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)

for i in range(n_bootstraps):
    indices = rng.randint(0, len(grr_pred), len(grr_pred))
    if len(np.unique(grr_true[indices])) < 2:
        continue

    score = roc_auc_score(grr_true[indices], grr_pred[indices])
    bootstrapped_scores.append(score)
    
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

grr_low = sorted_scores[int(0.05 * len(sorted_scores))]
grr_up = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.5f} - {:0.5}]".format(grr_low, grr_up))

In [ ]:
### Cross Validation: GUSTO Ridge Regression

In [ ]:
#Cross Validation with kfold
k = 5
kf = KFold(n_splits=k, random_state=None)
model1 = RidgeCV(alphas= r_alphas, cv = k)

result = cross_val_score(model1 , g_Xtest, g_Ytest, cv = kf)
print("Avg accuracy: {}".format(result.mean()))

In [ ]:
#Run RidgeCV with KFold=5

grr_cv = RidgeCV(cv=5)
grr_cv.fit(g_Xtrain,g_Ytrain)

grr_pred_cv = grr_cv.predict(g_Xtest)

gcv_auc_roc = roc_auc_score(g_Ytest, grr_pred_cv)
print("Training AUC: %f" % gcv_auc_roc)

In [ ]:
# Calculate Confidence Interval using bootstrap for GUSTO Ridge Regression model (with Kfold CV)

#Y train value for Logistic Regression model
grr_true_cv = np.array(g_Ytest)

#Predict value for Ridge Regression model
grr_pred_cv1 = np.array(grr_pred_cv)


n_bootstraps = 100
rng_seed = 42  # control reproducibility
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)

for i in range(n_bootstraps):
    indices = rng.randint(0, len(grr_pred_cv1), len(grr_pred_cv1))
    if len(np.unique(grr_true_cv[indices])) < 2:
        continue

    score = roc_auc_score(grr_true_cv[indices], grr_pred_cv1[indices])
    bootstrapped_scores.append(score)
    
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

grr_low_cv = sorted_scores[int(0.05 * len(sorted_scores))]
grr_up_cv = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.5f} - {:0.5}]".format(grr_low_cv, grr_up_cv))

### S2 Question 3: GUSTO Neural Net Model

In [ ]:
gmlp = MLPClassifier(
    hidden_layer_sizes=(100,80,10,1),
    activation='relu',
    max_iter=500,
    alpha=0.10,
    solver="sgd",
    verbose=10,
    random_state=1,
    learning_rate='adaptive',
    learning_rate_init=0.001,
)


gmlp.fit(g_Xtrain, g_Ytrain)

gmlp_predtrain = gmlp.predict_proba(g_Xtrain)
print("Train set AUC score: %f" % roc_auc_score(g_Ytrain, gmlp_predtrain[:,1]))


gmlp_predtest = gmlp.predict_proba(g_Xtest)
print("Train set AUC score: %f" % roc_auc_score(g_Ytest, gmlp_predtest[:,1]))

### Cross Validation: GUSTO Neural Net Model

In [ ]:
gmlp = MLPClassifier(max_iter=200) 
parameter_space = { 'hidden_layer_sizes': [(50,20,10,1),(50,10),(20,10,5), (100,80,10,1)], 
                   'activation': ['relu'], 
                   'solver': ['sgd','adam'], 
                   'batch_size': [300,500], 
                   'alpha': [0.10,0.40,0.60,0.80,1.0], 
                   'random_state': [1], 
                   'learning_rate': ['constant', 'adaptive'],
                   'learning_rate_init': [0.1, 0.01, 0.001] }

gclf = GridSearchCV(gmlp, parameter_space, n_jobs=-1, cv=5) 
gclf.fit(g_Xtrain, g_Ytrain) 

print('Best parameters found:\n', gclf.best_params_)

In [ ]:
#Calculate AUC after GridSearch 

gmlp2 = MLPClassifier(
    hidden_layer_sizes=(50,10),
    activation='relu',
    max_iter=100,
    alpha=0.60,
    solver="sgd",
    verbose=10,
    random_state=1,
    learning_rate='constant',
    learning_rate_init=0.01,
    batch_size = 500 
)


gmlp2.fit(g_Xtrain, g_Ytrain)

gmlp_predtrain2 = gmlp2.predict_proba(g_Xtrain)
print("Train set AUC score: %f" % roc_auc_score(g_Ytrain, gmlp_predtrain2[:,1]))

gmlp_predtest2 = gmlp2.predict_proba(g_Xtest)

In [ ]:
#Calculate Confidence Interval After Neural Net GridSearch GUSTO

y_true_g = np.array(g_Ytest)
y_pred_g = np.array(gmlp_predtest2[:,1])

n_bootstraps = 100
rng_seed = 42
bootstrapped_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    indices = rng.randint(0, len(y_pred_g), len(y_pred_g))
    if len(np.unique(y_true_g[indices])) < 2:
        continue

    score = roc_auc_score(y_true_g[indices], y_pred_g[indices])
    bootstrapped_scores.append(score)
    
sorted_scores = np.array(bootstrapped_scores)
sorted_scores.sort()

confidence_lower_nng = sorted_scores[int(0.05 * len(sorted_scores))]
confidence_upper_nng = sorted_scores[int(0.95 * len(sorted_scores))]
print("Confidence interval for the score: [{:0.5f} - {:0.5}]".format(confidence_lower_nng, confidence_upper_nng))